In [ ]:
crsts = '/home/jasp/autocls/exp4/class'

train = '/mnt/d/autocls_e4/trainref/cosc21_train_lvl_1.tif'

outfolder = '/home/jasp/autocls/exp4/study_l1'

In [ ]:
from osgeo import gdal, gdal_array
import numpy as np
import pandas as pd
import os
from glass.pys.oss import lst_ff, mkdir
from glass.pys.tm import now_as_str
from glass.wt.rst import obj_to_rst
from glass.wt import obj_to_tbl
from glass.prop.img import rst_epsg

In [ ]:
rsts = lst_ff(crsts, file_format='.tif')

ref = gdal.Open(train, gdal.GA_ReadOnly)

src = [gdal.Open(r, gdal.GA_ReadOnly) for r in rsts]

In [ ]:
tmp = src[0].GetRasterBand(1).ReadAsArray()

epsg = rst_epsg(ref)

gtrans = ref.GetGeoTransform()

In [ ]:
# Get NoData Values
nd_ref = ref.GetRasterBand(1).GetNoDataValue()
nd_cls = src[0].GetRasterBand(1).GetNoDataValue()

In [ ]:
all_rsts = np.zeros(
    (ref.RasterYSize, ref.RasterXSize, len(rsts)),
    gdal_array.GDALTypeCodeToNumericTypeCode(src[0].GetRasterBand(1).DataType)
)

for i in range(len(src)):
    all_rsts[:, :, i] = src[i].GetRasterBand(1).ReadAsArray()

rshp_rsts = all_rsts[:, :, :len(src)].reshape(
    all_rsts.shape[0] * all_rsts.shape[1],
    all_rsts.shape[2]
)

In [ ]:
tnum = ref.GetRasterBand(1).ReadAsArray()

tnum_rshp = tnum.reshape(tnum.shape[0] * tnum.shape[1])

In [ ]:
df = pd.DataFrame({'data' : list(rshp_rsts), 'refclass' : list(tnum_rshp)})

In [ ]:
def get_info(_x):
    v, counts = np.unique(_x['data'], return_counts=True)

    total = sum(list(counts))

    _x['nclasses'] = len(v)
    _x['classes'] = ", ".join(map(str, list(v)))

    majority = max(list(counts))

    otherp = 0
    ref_p = 0
    for i in range(len(v)):
        if v[i] == _x['refclass']:
            ref_p += counts[i]
        else:
            otherp += counts[i]

        if counts[i] == majority:
            majclss = v[i]

    _x['ref_p'] = ref_p / total * 100.0
    _x['other_p'] = otherp / total * 100.0
    _x['maj_class'] = majclss
    _x['maj_p'] = majority / total * 100.0
    
    return _x

In [ ]:
df = df.apply(lambda x: get_info(x), axis=1)

In [ ]:
d = {
    'nclasses'  : np.array(df['nclasses'], dtype=np.uint8).reshape(tmp.shape),
    'ref_p'     : np.array(df['ref_p'], dtype=np.uint8).reshape(tmp.shape),
    'other_p'   : np.array(df['other_p'], dtype=np.uint8).reshape(tmp.shape),
    'maj_class' : np.array(df['maj_class'], dtype=np.uint8).reshape(tmp.shape),
    'maj_p'     : np.array(df['maj_p'], dtype=np.uint8).reshape(tmp.shape)
}

In [ ]:
df.drop(['data'], axis=1, inplace=True)

In [ ]:
# Place NoData
for k in d:
    np.place(d[k], tnum == nd_ref, 255)
    np.place(d[k], tmp == nd_cls, 255)

In [ ]:
# Save results
mkdir(outfolder, overwrite=True)

obj_to_tbl(df, os.path.join(outfolder, 'exp4_ftable.xlsx'))


for k in d:
    obj_to_rst(
        d[k],
        os.path.join(outfolder, f'exp4_{k}.tif'),
        gtrans, epsg, noData=255
    )

In [ ]:
df

In [ ]:
d['nclasses'].dtype